In [2]:
# Importar librerias
import numpy as np
import pandas as pd
import ast
import json

In [3]:
# Lectura del dataset de películas
movies = pd.read_csv('../Datasets/movies_dataset.csv', low_memory=False)
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
# Eliminamos las columnas que no vamos a usar en el modelo ('adult', 'homepage', 'imdb_id', 'original_title', 'poster_path', 'video')
columns_drop = ['adult', 'homepage', 'imdb_id', 'original_title', 'poster_path', 'video']
movies = movies.drop(columns= columns_drop, axis=1)
movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0


In [5]:
# verificamos si el dataset tiene valores vacios
empty_values = (movies == '').sum()
empty_values

belongs_to_collection    0
budget                   0
genres                   0
id                       0
original_language        0
overview                 0
popularity               0
production_companies     0
production_countries     0
release_date             0
revenue                  0
runtime                  0
spoken_languages         0
status                   0
tagline                  0
title                    0
vote_average             0
vote_count               0
dtype: int64

In [6]:
# Verificamos la cantidad de regitros nulos por columna
print(movies.info())
movies.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [7]:
# Vamos a remplazar valores faltantes en las columnas budget y revenue
columns_fillna = ['budget', 'revenue']
for i in columns_fillna:
    movies[i] = movies[i].fillna(0)

movies[['budget', 'revenue']].isnull().sum()

budget     0
revenue    0
dtype: int64

In [8]:
# Eliminar filas con valores nulos en la columna release_date
movies = movies.dropna(subset=['release_date'])

In [9]:
movies['release_date'].isnull().sum()

np.int64(0)

# Creamos la columna 'release_year'

In [10]:
# Verificamos que la columna 'relase_date' sea de tipo datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Creamos la columna release_year a partir de release_date
movies['release_year'] = movies['release_date'].dt.year

# Verificar el resultado
print(movies[['release_date', 'release_year']].head())

  release_date  release_year
0   1995-10-30        1995.0
1   1995-12-15        1995.0
2   1995-12-22        1995.0
3   1995-12-22        1995.0
4   1995-02-10        1995.0


In [11]:
# Aqui camobiamoss el tipo de dato de la columna release_year para eliminar el cero ya que era de type float
movies['release_year'] = movies['release_year'].astype('Int64')
movies['release_year'].head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: Int64

# Creamos la columna return
* calculamos el retorno de inversion de cada pelicula

In [12]:
# Asegúrate de que las columnas revenue y budget estén en formato numérico
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Creamos la columna return
movies['return'] = np.where((movies['budget'] == 0) | (movies['budget'].isna()), 0, movies['revenue'] / movies['budget'])

# Verificar el resultado
print(movies[['revenue', 'budget', 'return']].head())

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


# Desanidamos las columnas que poseen diccionarios y las volvemos un dataframe individual
* Columna 'belongs_to_collection'

In [13]:
# Asegurarnos de que los valores en 'belongs_to_collection' sean diccionarios, si no, parsearlos
def ensure_dict(x):
    if isinstance(x, str):
        return ast.literal_eval(x)  # Solo evaluamos si es una cadena
    return x  # Si ya es un diccionario, lo dejamos como está

In [14]:
# Función para manejar errores en JSON
def safe_json_loads(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return []

In [15]:
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: ensure_dict(x) if pd.notnull(x) else None)

# Convertir cada elemento de la columna 'belongs_to_collection' en un DataFrame
# Si el valor es None, devolvemos un DataFrame vacío
belong = movies['belongs_to_collection'].apply(lambda x: pd.DataFrame([x]) if pd.notnull(x) else pd.DataFrame())

# Asignar un índice para cada fila en el DataFrame original
movies['index'] = movies.index

# Combinar todos los DataFrames individuales en uno solo
column_belong = pd.concat([df.assign(index=idx) for idx, df in belong.items()], ignore_index=True)


In [16]:
# Eliminamos la columna '0'
column_belong = column_belong.drop(columns=[0, 'index', 'poster_path', 'backdrop_path'])

*Columna 'belongs_to_collection' Un diccionario que indica a qué franquicia o serie de películas pertenece la película. Incluye id, name, poster_path y backdrop_path.*

In [17]:
column_belong.head(3)

,id,name
0,10194.0,Toy Story Collection
1,119050.0,Grumpy Old Men Collection
2,96871.0,Father of the Bride Collection


* Columna 'genres'

In [18]:
# Cambiar el nombre de la columna id por id_movie
movies = movies.rename(columns={'id': 'id_movie'})

In [19]:
# Supongamos que 'movies' es el DataFrame original con una columna 'genres'

# Asegúrate de que 'genres' en 'movies' sea una lista de diccionarios o NaN
movies['genres'] = movies['genres'].apply(lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x)
movies['original_index'] = movies.index

# Desanidar 'genres' y mantener el índice original
column_genres = movies.explode('genres')
column_genres['original_index'] = column_genres['original_index']

# Extraer campos del género y manejar NaN
column_genres['genre_id'] = column_genres['genres'].apply(lambda x: x['id'] if isinstance(x, dict) else np.nan)
column_genres['genre_name'] = column_genres['genres'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

# Eliminar la columna 'genres' que ya no es necesaria
column_genres = column_genres.drop(columns=['genres'])

# Preparar el DataFrame original eliminando la columna 'genres'
movies_cleaned = movies.drop(columns=['genres'])

# Combinar con el DataFrame desanidado 'column_genres' usando 'original_index'
movies_expanded = pd.merge(movies_cleaned, column_genres, on='original_index', how='left')

# Eliminar la columna auxiliar 'original_index'
movies_expanded = movies_expanded.drop(columns=['original_index'])

In [20]:
movies_expanded = movies_expanded.drop(['belongs_to_collection_y', 'production_companies_y', 'production_countries_y', 'spoken_languages_y', 'status_y', 'tagline_y', 'title_y', 'original_language_y', 'overview_y', 'popularity_y', 'release_date_y', 'revenue_y', 'runtime_y', 'vote_average_y', 'vote_count_y', 'release_year_y', 'return_y', 'index_x', 'index_y', 'id_movie_y', 'budget_y'], axis=1)

In [21]:
columns_name = {
    "id_movie_x": "id_movie",
    "title_x": "title",
    "release_date_x": "release_date",
    "popularity_x": "popularity",
    "vote_average_x": "vote_average",
    "vote_count_x": "vote_count",
    "overview_x": "overview",
    "original_language_x": "original_language",
    "original_title_x": "original_title",
    "runtime_x": "runtime",
    "tagline_x": "tagline",
    "budget_x": "budget",
    "revenue_x": "revenue",
    "production_companies_x": "production_companies",
    "production_countries_x": "production_countries",
    "spoken_languages_x": "spoken_languages",
    "return_x": "return",
    "release_year_x": "release_year",
    "status_x": "status",
    "belongs_to_collection_x": "belongs_to_collection",
}

In [22]:
# Renombrar columnas utilizando el diccionario
movies_expanded = movies_expanded.rename(columns=columns_name)

In [23]:
movies_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93432 entries, 0 to 93431
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  11156 non-null  object        
 1   budget                 93420 non-null  float64       
 2   id_movie               93432 non-null  object        
 3   original_language      93415 non-null  object        
 4   overview               92116 non-null  object        
 5   popularity             93424 non-null  object        
 6   production_companies   93432 non-null  object        
 7   production_countries   93432 non-null  object        
 8   release_date           93420 non-null  datetime64[ns]
 9   revenue                93432 non-null  float64       
 10  runtime                93099 non-null  float64       
 11  spoken_languages       93420 non-null  object        
 12  status                 93296 non-null  object        
 13  t

In [24]:
"""
# Reemplazar valores nulos por una lista vacía
movies['genres'] = movies['genres'].fillna("[]")

# Primero, reemplaza comillas simples por comillas dobles para que sea un JSON válido
movies['genres'] = movies['genres'].str.replace("'", '"')

# Convertir la columna de texto a una lista de diccionarios
movies['genres'] = movies['genres'].apply(json.loads)

# Desanidar la columna
column_genres = movies['genres'].explode().apply(pd.Series)
"""

'\n# Reemplazar valores nulos por una lista vacía\nmovies[\'genres\'] = movies[\'genres\'].fillna("[]")\n\n# Primero, reemplaza comillas simples por comillas dobles para que sea un JSON válido\nmovies[\'genres\'] = movies[\'genres\'].str.replace("\'", \'"\')\n\n# Convertir la columna de texto a una lista de diccionarios\nmovies[\'genres\'] = movies[\'genres\'].apply(json.loads)\n\n# Desanidar la columna\ncolumn_genres = movies[\'genres\'].explode().apply(pd.Series)\n'

In [25]:
movies_df = column_genres.copy()

*Un diccionario que indica todos los géneros asociados a la película. Incluye id y name.*

In [26]:
movies_df.head(2)

,belongs_to_collection,budget,id_movie,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,tagline,title,vote_average,vote_count,release_year,return,index,original_index,genre_id,genre_name
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,NaN,Toy Story,7.7,5415.0,1995,12.451801,0,0,16.0,Animation
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,NaN,Toy Story,7.7,5415.0,1995,12.451801,0,0,35.0,Comedy


* Columna 'production_companies'

In [27]:
# Reemplazar valores nulos por una lista vacía
movies_expanded['production_companies'] = movies_expanded['production_companies'].fillna("[]")

# Reemplazar comillas simples por comillas dobles para que sea un JSON válido
movies_expanded['production_companies'] = movies_expanded['production_companies'].str.replace("'", '"')

# Convertir la columna de texto a una lista de diccionarios, manejando errores
movies_expanded['production_companies'] = movies_expanded['production_companies'].apply(safe_json_loads)

# Desanidar la columna
column_production = movies_expanded['production_companies'].explode().apply(pd.Series)

In [28]:
# Eliminamos la columna '0'
column_production = column_production.drop(columns=[0])

*Lista con las compañías productoras asociadas a la película. Incluye name e id.*

In [29]:
column_production.head()

,name,id
0,Pixar Animation Studios,3.0
1,Pixar Animation Studios,3.0
2,Pixar Animation Studios,3.0
3,TriStar Pictures,559.0
3,Teitler Film,2550.0


* Columna 'production_countries'

In [30]:
movies_expanded['production_countries'] = movies_expanded['production_countries'].fillna("[]")
movies_expanded['production_countries'] = movies_expanded['production_countries'].str.replace("'", '"')
movies_expanded['production_countries'] = movies_expanded['production_countries'].apply(safe_json_loads)
column_countries = movies_expanded['production_countries'].explode().apply(pd.Series)

In [31]:
# Eliminamos la columna '0'
column_countries = column_countries.drop(columns=[0])

*Lista con los países donde se produjo la película. Incluye iso_3166_1 y name.*

In [32]:
column_countries.head()

,iso_3166_1,name
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America


* Columna 'spoken_languages'

In [33]:
movies['spoken_languages'] = movies['spoken_languages'].fillna("[]")
movies['spoken_languages'] = movies['spoken_languages'].str.replace("'", '"')
movies['spoken_languages'] = movies['spoken_languages'].apply(safe_json_loads)
column_languages = movies['spoken_languages'].explode().apply(pd.Series)

In [34]:
# Eliminamos la columna '0'
column_languages = column_languages.drop(columns=[0])

*Lista con los idiomas que se hablan en la película. Incluye iso_639_1 y name.*

In [35]:
column_languages.head()

,iso_639_1,name
0,en,English
1,en,English
1,fr,Français
2,en,English
3,en,English


In [36]:
# Procede a borrar las columnas que tenian diccionarios y los que no vamos a usar
movies_expanded = movies_expanded.drop(columns=['belongs_to_collection', 'production_companies', 'production_countries', 'spoken_languages', 'original_language', 'overview', 'status', 'tagline'])
movies_expanded.head(2)

,budget,id_movie,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name
0,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation
1,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,35.0,Comedy


### Terminamos de limpiar antes de exportarlo

In [38]:
# Asegúrate de que las columnas popularity esté en formato numérico
movies_expanded['popularity'] = pd.to_numeric(movies_expanded['popularity'], errors='coerce')
# Rellenar valores nulos en cada columna
movies_expanded['budget'] = movies_expanded['budget'].fillna(movies_expanded['budget'].mean())
movies_expanded['popularity'] = movies_expanded['popularity'].fillna(movies_expanded['popularity'].median())
movies_expanded['release_date'] = movies_expanded['release_date'].fillna(movies_expanded['release_date'].median())
movies_expanded['runtime'] = movies_expanded['runtime'].fillna(movies_expanded['runtime'].median())
movies_expanded['title'] = movies_expanded['title'].fillna('Desconocido')
movies_expanded['vote_average'] = movies_expanded['vote_average'].fillna(movies_expanded['vote_average'].mean())
movies_expanded['vote_count'] = movies_expanded['vote_count'].fillna(movies_expanded['vote_count'].median())
movies_expanded['release_year'] = movies_expanded['release_year'].fillna(movies_expanded['release_year'].median())
movies_expanded['genre_id'] = movies_expanded['genre_id'].fillna(0)
movies_expanded['genre_name'] = movies_expanded['genre_name'].fillna('Desconocido')

In [39]:
movies_expanded['genre_name'] = movies_expanded['genre_name'].astype(str)

In [40]:
movies_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93432 entries, 0 to 93431
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        93432 non-null  float64       
 1   id_movie      93432 non-null  object        
 2   popularity    93432 non-null  float64       
 3   release_date  93432 non-null  datetime64[ns]
 4   revenue       93432 non-null  float64       
 5   runtime       93432 non-null  float64       
 6   title         93432 non-null  object        
 7   vote_average  93432 non-null  float64       
 8   vote_count    93432 non-null  float64       
 9   release_year  93432 non-null  Int64         
 10  return        93432 non-null  float64       
 11  genre_id      93432 non-null  float64       
 12  genre_name    93432 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(8), object(3)
memory usage: 9.4+ MB


### Exportamos los dataframes a csv

In [ ]:
movies_expanded.to_parquet('../Datasets/movies.parquet', index=False, compression='snappy')

In [ ]:
#movies_expanded.to_csv('../Datasets/movies.csv', index=False)
#column_belong.to_csv('../Datasets/column_belong.csv', index=False, compression='gzip')
#column_genres.to_csv('../Datasets/column_genres.csv', index=False, compression='gzip')
#column_countries.to_csv('../Datasets/column_countries.csv', index=False, compression='gzip')
#column_production.to_csv('../Datasets/column_production.csv', index=False, compression='gzip')
#column_languages.to_csv('../Datasets/column_languages.csv', index=False, compression='gzip')

In [56]:
title = movies_expanded.loc[movies_expanded['title'].str.lower() == 'Toy Story 2']

In [57]:
title

,budget,id_movie,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name
